In [1]:
!pip install -q -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.1 MB/s eta 0:00:00


In [2]:
!pip install -q transformers torch huggingface_hub accelerate

In [3]:
import sqlite3
import pandas as pd
import re

conn = sqlite3.connect("employee.db")
cursor = conn.cursor()

In [4]:
def load_csv_data(csv_file):
    df = pd.read_csv(csv_file)

    columns = df.columns
    column_types = ", ".join([f'"{col}" TEXT' for col in columns])
    cursor.execute(f"CREATE TABLE IF NOT EXISTS employee ({column_types});")

    df.to_sql("employee", conn, if_exists="append", index=False)
    conn.commit()
    print("CSV data uploaded successfully!")

In [5]:
def execute_query(query):
    cursor.execute(query)
    rows = cursor.fetchall()
    return rows

In [6]:
load_csv_data("employee_data.csv")
# execute_query("select * from employee;")

CSV data uploaded successfully!


In [7]:
#LLAMA 3.1 8B CONNECTIONS
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import os

model_id = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer,
                          max_new_tokens=128)

def get_response(prompt):
    response = text_generator(prompt)
    return response[0]['generated_text']

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
#PROMPT
def create_sql_prompt(table_name, user_query):
    """
    Create a prompt for the SQL query agent.

    Args:
        table_name (str): The name of the SQL table.
        user_query (str): The user's natural language query.

    Returns:
        str: The formatted prompt for the model.
    """
    schema = f"""
    You are an SQL query generator. Given the following SQL table schema, generate a SQL query based on the user's request.
    Table: {table_name}
    Columns:
    - EmployeeID (INTEGER): Unique ID
    - Name (TEXT): Random full name
    - Age (INTEGER): Between 22 and 60
    - Gender (TEXT): Male/Female
    - Department (TEXT): e.g., HR, IT, Sales, Finance, etc.
    - Salary (INTEGER): Random salary between $30,000 and $150,000
    - JoiningDate (DATE): Random date within the last 10 years
    - ExperienceYears (INTEGER): Between 0 and 40
    - PerformanceRating (INTEGER): Scale of 1-5
    - Resigned (TEXT): Yes/No

    User Query: "{user_query}"
    """

    return schema + "\nGenerate the SQL query:"

In [9]:
table_name = "employee"  # Example table name
user_query = input("Enter your query: ")  # Get user input
# user_query = "total number of female employees"
# Create SQL prompt
sql_prompt = create_sql_prompt(table_name, user_query)

# Get SQL query from the model
sql_query = get_response(sql_prompt)
print("Generated SQL Query:", sql_query)

# Query the database
print("Querying the database...")

# matches = re.findall(r'```sql\n.*\n```',sql_query)[-1].split("\n")[1]

# execute_query(matches)
final_query = re.search(r'SELECT.*', sql_query).group()

Enter your query: total number of male employees
Generated SQL Query: 
    You are an SQL query generator. Given the following SQL table schema, generate a SQL query based on the user's request.
    Table: employee
    Columns:
    - EmployeeID (INTEGER): Unique ID
    - Name (TEXT): Random full name
    - Age (INTEGER): Between 22 and 60
    - Gender (TEXT): Male/Female
    - Department (TEXT): e.g., HR, IT, Sales, Finance, etc.
    - Salary (INTEGER): Random salary between $30,000 and $150,000
    - JoiningDate (DATE): Random date within the last 10 years
    - ExperienceYears (INTEGER): Between 0 and 40
    - PerformanceRating (INTEGER): Scale of 1-5
    - Resigned (TEXT): Yes/No

    User Query: "total number of male employees"
    
Generate the SQL query: 
SELECT COUNT(*) FROM employee WHERE Gender = 'Male'

```sql
SELECT COUNT(*) FROM employee WHERE Gender = 'Male'
```


Explanation:

The user wants to know the total number of male employees. To achieve this, we need to count the

In [10]:
final_query

"SELECT COUNT(*) FROM employee WHERE Gender = 'Male'"

In [11]:
execute_query(final_query)

[(10051,)]